Initialize problem

In [ ]:
import logging
from collections import namedtuple
import random
from typing import Callable
from copy import deepcopy
from itertools import accumulate
from operator import xor

In [ ]:
Nimply = namedtuple("Nimply", "row, num_objects")

In [ ]:
class Nim:
    def __init__(self, num_rows: int, k: int = None) -> None:
        self._rows = [i * 2 + 1 for i in range(num_rows)]
        self._k = k
        self._active_rows = [i for i in range(num_rows)]

    def __bool__(self):
        return sum(self._rows) > 0

    def __str__(self):
        return "<" + " ".join(str(_) for _ in self._rows) + ">"

    @property
    def rows(self) -> tuple:
        return tuple(self._rows)

    @property
    def k(self) -> int:
        return self._k

    def active_rows(self) -> list:
        return self._active_rows

    def nimming(self, ply: Nimply) -> None:
        row, num_objects = ply
        assert self._rows[row] >= num_objects
        assert self._k is None or num_objects <= self._k
        self._rows[row] -= num_objects
        if self._rows[row] == 0:
            self._active_rows.remove(row)

Starting task 1

In [ ]:
def pure_random(state: Nim) -> Nimply:
    row = random.choice([r for r, c in enumerate(state.rows) if c > 0])
    num_objects = random.randint(1, state.rows[row])
    return Nimply(row, num_objects)

def optimal_strategy(board: Nim) -> Nimply: 
    *_, X = accumulate(board.rows, xor)

    row_count = -1
    for row in board.rows:
        row_count += 1
        if row > 0:
            result = X ^ row
            if result < row:
                return Nimply(row_count, row - result)
    return pure_random(board)

In [ ]:
logging.getLogger().setLevel(logging.DEBUG)

strategy = (optimal_strategy, optimal_strategy)

nim = Nim(11)
logging.debug(f"status: Initial board  -> {nim}")
player = 0
while nim:
    ply = strategy[player](nim)
    nim.nimming(ply)
    logging.debug(f"status: After player {player} -> {nim}")
    player = 1 - player
winner = 1 - player
logging.info(f"status: Player {winner} won!")

Starting task 2

In [ ]:
class Rules: 
    def __init__(self, board, rule_1, rule_2, rule_3, rule_4):
        max_leave_nim_size = (len(board.rows) - 1) * 2

        self.rules = {}

        # How many elements to leave from the only active row
        if rule_1 <= max_leave_nim_size:
            self.rules['rule_1'] = rule_1
        else:
            self.rules['rule_1'] = random.randint(0, max_leave_nim_size)
        
        # How many elements to leave from the selected row of the two active rows,
        # where one row has only 1 element left
        if rule_2[1] <= max_leave_nim_size:
            self.rules['rule_2'] = rule_2
        else:
            self.rules['rule_2'] = (random.randint(0, 1), random.randint(0, max_leave_nim_size))

        # How many elements to leave from the selected row of the two active rows, 
        # where both rows have more than 1 element left
        if rule_3[1] <= max_leave_nim_size:
            self.rules['rule_3'] = rule_3
        else:
            self.rules['rule_3'] = (random.randint(0,1), random.randint(0, max_leave_nim_size))

        # If there are more than two rows left, leave x elements from a random row
        if rule_4 <= max_leave_nim_size:
            self.rules['rule_4'] = rule_4
        else:
            self.rules['rule_4'] = random.randint(0, max_leave_nim_size)

    def get_params(self):
        return self.rules

    def __feasability__(self, x, number_of_elements, row_index):
        if number_of_elements - 1 >= x:
            return Nimply(row_index, number_of_elements - x)
        else:
            return Nimply(row_index, 1)
    
    def set_rule_1(self, val):
        self.rules['rule_1'] = val

    def set_rule_2(self, val):
        row, _ = self.rules['rule_2']
        self.rules['rule_2'] = (row, val)

    def set_rule_3(self, val):
        row, _ = self.rules['rule_3']
        self.rules['rule_3'] = (row, val)

    def set_rule_4(self, val):
        self.rules['rule_4'] = val

    # If there is only 1 active row left, leave x elements in that row
    def rule_1(self, board):
        # Obtain the last row index and the number of elements in the row
        active_rows = board.active_rows()
        
        number_of_elems = board.rows[active_rows[0]]

        # Check that the number of elements we want to leave behind is feasible, otherwise
        # we pick a single element
        return self.__feasability__(self.rules['rule_1'], number_of_elems, active_rows[0])
                

    # If there are two rows left, where one row has onle 1 elemnt left, leave x elements
    # in a random row
    def rule_2(self, board):
        return self.__find_active_rows__(board, 'rule_2')


    # If there are two rows left where both rows have more than 1 element left, leave x elements
    # in a random row
    def rule_3(self, board):
        return self.__find_active_rows__(board, 'rule_3')

    def __find_active_rows__(self, board, rule):
        active_rows = board.active_rows()
        rows_and_elements = []

        for index in active_rows:
            rows_and_elements.append([index, board.rows[index]])

        # Sort the list so the smaller row is first 
        rows_and_elements.sort(key=lambda x: x[1])
        
        # Determine if the number of elements we want to leave behind is feasible, otherwise 
        # pick a single element
        chosen_row = rows_and_elements[self.rules[rule][0]]
        return self.__feasability__(self.rules[rule][1], chosen_row[1], chosen_row[0])

    # If there are more than two rows left, leave x elements in a random row
    def rule_4(self, board):
        active_rows = board.active_rows()
        rows_and_elements = []

        for index in active_rows:
            rows_and_elements.append([index, board.rows[index]])

        chosen_row = random.choice(rows_and_elements)
        return self.__feasability__(self.rules['rule_4'], chosen_row[1], chosen_row[0])

In [ ]:
def make_strategy(individual: Rules) -> Callable:
    def evolvable(board: Nim) -> Nimply:
        active_rows = board.active_rows()
        rows_and_elements = []

        for index in active_rows:
            rows_and_elements.append([index, board.rows[index]])

        # Sort the list so the smaller row is first 
        rows_and_elements.sort(key=lambda x: x[1])
        
        # If there's only 1 row left
        if len(rows_and_elements) == 1:
            ply = individual.rule_1(board)
        # If there's two rows left
        elif len(rows_and_elements) == 2:
            # If both active rows have more than 1 element
            if rows_and_elements[0][1] > 1 and rows_and_elements[0][1] > 1:
                ply = individual.rule_3(board)
            else:
                ply = individual.rule_2(board)
        # If there's more than two rows left
        elif len(rows_and_elements) > 2:
            ply = individual.rule_4(board)
        return ply

    return evolvable

def pure_random(state: Nim) -> Nimply:
    row = random.choice([r for r, c in enumerate(state.rows) if c > 0])
    num_objects = random.randint(1, state.rows[row])
    return Nimply(row, num_objects)

def optimal_strategy(board: Nim) -> Nimply: 
    *_, X = accumulate(board.rows, xor)

    row_count = -1
    for row in board.rows:
        row_count += 1
        if row > 0:
            result = X ^ row
            if result < row:
                return Nimply(row_count, row - result)
    return pure_random(board)

def stupid(board: Nim) -> Nimply:
    return Nimply(board.active_rows()[0], 1)


In [ ]:
def stupid(board: Nim) -> Nimply:
    return Nimply(board.active_rows()[0], 1)

In [ ]:
def fitness(individual, fit_rounds, depth):
    fit = []
    for opponent in [optimal_strategy, pure_random, stupid]:
        wins = 0
        strategy = (make_strategy(individual), opponent)
        for _ in range(fit_rounds):
            wins += gameplay(strategy, depth)
        fit.append(wins)
    return (fit[0], fit[1], fit[2])

def gameplay(strategy, depth):
    player = 0
    nim = Nim(depth)
    while nim:
        ply = strategy[player](nim)
        nim.nimming(ply)
        player = 1 - player
    winner = 1 - player
    if winner == 0:
        return 1
    else:
        return 0

def population_init(depth, mu, fit_rounds):
    individuals = []
    max_leave_nim_size = (depth - 1) * 2

    for _ in range(mu):
        individual = Rules(Nim(depth), random.randint(0, max_leave_nim_size), \
            (random.randint(0, 1), random.randint(0, max_leave_nim_size)), \
            (random.randint(0,1), random.randint(0, max_leave_nim_size)), \
            random.randint(0, max_leave_nim_size))
        fit = fitness(individual, fit_rounds, depth)
        individuals.append([fit, individual])
    return individuals

def tournament(individuals, offspring_size, depth, mut_probability, fit_rounds):
    offspring = []
    for _ in range(offspring_size):
        p1 = max(random.choices(individuals, k=10), key=lambda x: x[0])
        p2 = max(random.choices(individuals, k=10), key=lambda x: x[0])

        offspring_individual = crossover(p1, p2, depth, mut_probability)
        fitness_offspring = fitness(offspring_individual, fit_rounds, depth)

        offspring.append((fitness_offspring, offspring_individual))
    return offspring

def crossover(p1, p2, depth, mut_probability):
    # Chose a random amount of rules of the first parent and the rest from the second parent
    elements = random.randint(0,4)
    n1 = random.sample([i+1 for i in range(4)], elements)
    # Pick the rules from the second parent that the first parent didnt pick
    n2 = list(set(n1) ^ set([1, 2, 3, 4]))

    rules = []
    for rule_number in n1:
        rules.append((rule_number, 0))
    
    for rule_number in n2:
        rules.append((rule_number, 1))
    
    parents = []
    parents.append(p1[1].get_params())
    parents.append(p2[1].get_params())
    
    rules.sort()

    # Create the offspring
    offspring = Rules(Nim(depth), parents[rules[0][1]]['rule_1'], parents[rules[1][1]]['rule_2'], \
    parents[rules[2][1]]['rule_3'], parents[rules[3][1]]['rule_4'])

    # With a certain probability the offpsring is mutated
    if random.random() < mut_probability:
        mutate_rule = random.randint(1, 4)
        max_leave_nim_size = (depth - 1) * 2
        if mutate_rule == 1:
            offspring.set_rule_1(random.randint(0, max_leave_nim_size))
        elif mutate_rule == 2:
            offspring.set_rule_2(random.randint(0, max_leave_nim_size))
        elif mutate_rule == 3:
            offspring.set_rule_3(random.randint(0, max_leave_nim_size))
        elif mutate_rule == 4:
            offspring.set_rule_4(random.randint(0, max_leave_nim_size))
    return offspring

In [ ]:
mu = 20
offspring_size = 40
fit_rounds = 10
mut_probability = 0.1
depth = 10

individuals = population_init(depth, mu, fit_rounds)
individuals.sort(key = lambda x: x[0], reverse=True)
old_best = individuals[0][1]

for _ in range(500):
    offspring = tournament(individuals, offspring_size, depth, mut_probability, fit_rounds)
    offspring.sort(key = lambda x: x[0], reverse=True)

    individuals = offspring[0:mu]
    if _ %50 == 0:
        print("Progress")
    
    if _ %100 == 0:
        if individuals[0][1] == old_best:
            print('oh oh')
            break
        else:
            old_best = individuals[0][1]

In [ ]:
individual = individuals[0][1]
strategy = (make_strategy(individual), pure_random)
wins = 0

for _ in range(1000):
    nim = Nim(15)
    #print(f"status: Initial board  -> {nim}")
    player = 0
    while nim:
        ply = strategy[player](nim)
        nim.nimming(ply)
        #print(f"status: After player {player} -> {nim}")
        player = 1 - player
    winner = 1 - player
    if winner == 0:
        wins += 1
print(wins)

In [ ]:
print(individual.get_params())

In [ ]:
nim = Nim(5)
individual = individuals[0][1]
strategy = (make_strategy(individual), optimal_strategy)

player = 0
while nim:
    print(nim)
    if player == 0:
        ply = (int(input()), int(input()))
    else:
        ply = strategy[1](nim)
    nim.nimming(ply)
    player = 1 - player
winner = 1 - player
if winner == 0:
    print("Player 0 won")
else:
    print("Player 1 won")


Starting task 3

In [ ]:
def best_move(nim):
    row_count = -1
    
    for row in nim.rows:
        row_count += 1
        for i in range(row):
            temp_state = deepcopy(nim)
            temp_state.nimming((row_count, i+1))
            score = evaluate(temp_state, 1)
            if score > 0:
                return score, (row_count, i+1)
    return score, (row_count, i+1)

def evaluate(nim, player, alpha=-1, beta=1):
    if(not nim):
        temp = 1
        if player == 0:
            temp = -1
        return temp

    possible_states_list = []
    explored_states = set()
    row_count = -1
    for row in nim.rows:
        row_count += 1
        if row not in explored_states:
            explored_states.add(row)
            for i in range(row):
                temp_state = deepcopy(nim)
                temp_state.nimming((row_count, i+1))
                possible_states_list.append(temp_state)

    scores = []
    for new_state in possible_states_list:
        score = evaluate(new_state, 1 - player, alpha, beta)
        scores.append(score)
        if player == 0:
            alpha = max(alpha, score)
        else: 
            beta = min(beta, score)
        if beta <= alpha:
            break
    return (max if player == 0 else min)(scores)
        

In [ ]:
nim = Nim(3)
strategy = (best_move)

player = 1
while nim:
    print(nim)
    if player == 0:
        ply = (int(input()), int(input()))
    else:
        score, ply = strategy(nim)
        print(score)
    nim.nimming(ply)
    player = 1 - player
winner = 1 - player
if winner == 0:
    print("Player 0 won")
else:
    print("Player 1 won")

Starting task 4

In [295]:
class Qlearning():

    reward = 1
    penalty = -1
    
    def __init__(self, learning_rate, discount_rate, exploring_rate, previous_state = None, previous_move = None):
        self.q = {}
        self.learning_rate = learning_rate
        self.discount_rate = discount_rate
        self.exploring_rate = exploring_rate
        self.previous_state = previous_state
        self.previous_move = previous_move

    def set_learning_rate(self, learning_rate):
        self.learning_rate = learning_rate

    def set_discount_rate(self, discount_rate):
        self.discount_rate = discount_rate

    def set_exploring_rate(self, exploring_rate):
        self.exploring_rate = exploring_rate

    def state_action(self, current_state):
        possible_move_list = possible_moves(current_state)

        for move in possible_move_list:
            if (current_state.rows, move) not in self.q:
                self.q[(current_state.rows, move)] = random.uniform(0, 0.01)

    def policy(self, current_state):
        possible_move_list = possible_moves(current_state)

        if random.random() > self.exploring_rate:
            # Do not explore, find the best move
            chosen_move_val = self.q[(current_state.rows, possible_move_list[0])]
            chosen_move = possible_move_list[0]
            for move in possible_move_list[1:]:
                if self.q[(current_state.rows, move)] > chosen_move_val:
                    chosen_move = move
                    chosen_move_val = self.q[(current_state.rows, move)]
        else: 
            # We explore
            chosen_move = random.sample(possible_move_list, 1)[0]
        return chosen_move

    def update_policy(self, current_state):
        if not current_state:
            # Game is finished, update policy about the loss
            self.q[(self.previous_state.rows, self.previous_move)] += \
                self.learning_rate * (self.penalty - self.q[(self.previous_state.rows, self.previous_move)])
            current_move = self.previous_state = self.previous_move = None
        else: 
            self.state_action(current_state)
            current_move = self.policy(current_state)

            if self.previous_move != None:
                next_state = deepcopy(current_state)
                next_state.nimming(current_move)

                reward = 0
                if not next_state:
                    reward = 1

                possible_move_list = possible_moves(current_state) 
                
                max_Q_value = self.q[(current_state.rows, possible_move_list[0])]
                for move in possible_move_list[1:]:
                    if self.q[(current_state.rows, move)] > max_Q_value:
                        max_Q_value = self.q[(current_state.rows, move)]

                self.q[(self.previous_state.rows, self.previous_move)] += \
                    self.learning_rate * (reward + self.discount_rate * max_Q_value) \
                    - self.q[(self.previous_state.rows, self.previous_move)]

            self.previous_state = deepcopy(current_state)
            self.previous_move = current_move

        return current_move
            
def train_agent(depth):
    iterations = 1000
    exploring_increase = 0.3
    exploring_decrease = 0.5
    exploring_rate = 0.8
    agent = Qlearning(0.9, 0.25, exploring_rate)

    opponent_strategies = (stupid, pure_random, optimal_strategy)

    for opponent in opponent_strategies:
        for _ in range(iterations):
            play_game(depth, agent, opponent)
            exploring_rate -= exploring_decrease/iterations
        
            if exploring_rate < 0.01:
                exploring_rate = 0.01
            
            agent.set_exploring_rate(exploring_rate)
        exploring_rate += exploring_increase
        exploring_decrease -= 0.05
    return agent

def evaluate(agent):
    agent.set_exploring_rate(0)
    iterations = 100
    wins = [0, 0, 0]
    i = -1

    opponent_strategies = (stupid, pure_random, optimal_strategy)

    for opponent in opponent_strategies:
        i += 1
        for _ in range(iterations):
            nim = Nim(5)
            player = 0
            while nim:
                if player == 0:
                    try:
                        move = agent.policy(nim)
                    except KeyError:
                        move = agent.update_policy(nim)
                else: 
                    move = opponent(nim)
                nim.nimming(move)
                player = 1 - player
            wins[i] += player #winrate
    return wins

def possible_moves(nim):
    possible_moves_list = []
    row_count = -1
    for row in nim.rows:
        row_count += 1
        for i in range(row):
            possible_moves_list.append((row_count, i+1))
    return possible_moves_list

def play_game(depth, agent, opponent):
    nim = Nim(depth)
    player = 0

    while True: 
        if len(nim.active_rows()) == 0:
            return 1-player
        if player == 0:
            move = agent.update_policy(nim)
        else: 
            move = opponent(nim)
        nim.nimming(move)
        player = 1 - player

In [296]:
agent = train_agent(5)
print(agent.exploring_rate)

wins = evaluate(agent)

print(wins)

'''player = 0
while nim:
    print(nim)
    if player == 0:
        ply = (int(input()), int(input()))
    else:
        ply = agent.update_policy(nim)
    nim.nimming(ply)
    player = 1 - player
winner = 1 - player'''

0.050000000000030465
[100, 72, 0]


'player = 0\nwhile nim:\n    print(nim)\n    if player == 0:\n        ply = (int(input()), int(input()))\n    else:\n        ply = agent.update_policy(nim)\n    nim.nimming(ply)\n    player = 1 - player\nwinner = 1 - player'